In [ ]:
#modulo de manejo de datos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#modulos de graficas
import seaborn as sns; sns.set()
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

In [ ]:
datos = pd.read_csv('../input/winemag-data-130k-v2.csv')

In [ ]:
#visualizar los primeros 5 registros
datos.head(20)

In [ ]:
datos.info()

In [ ]:
datos.describe()

In [ ]:
datos[['country','variety','winery','taster_name','taster_twitter_handle','province','region_1','region_2']].describe()

In [ ]:
datos.country.value_counts().head(15).plot.barh(width=0.9,figsize=(10,6),color='darkred');

In [ ]:
#porcentaje de vinos por cada pais
(datos.country.value_counts(normalize=True)*100).head(6)

In [ ]:
datos.variety.value_counts().head(70)

In [ ]:
#Analizando US por tener el mayor %
US = datos[datos['country'] == 'US']

In [ ]:
US.head()

In [ ]:
years = US.title.str.extract('([1-2][0-9]{3})').astype(float)
years[years < 1990] = None
US = US.assign(year = years)
US=US.dropna(subset=['price'])

In [ ]:
#En la data principal Pinot Noir es la variedad mas 
plt.scatter(x=US[US['variety'] == 'Pinot Noir']['points'],y=US[US['variety'] == 'Pinot Noir']['price'],c=US[US['variety'] == 'Pinot Noir']['year']);

In [ ]:
US[US['variety'] == 'Pinot Noir']

In [ ]:
sns.boxplot(x='variety', y='year', data = US[US['variety'] == 'Pinot Noir'])

In [ ]:
sns.jointplot(x='year',y='price',data=US[US['variety'] == 'White Blend']);

In [ ]:
US = US.drop_duplicates('description')

In [ ]:
US.shape

In [ ]:
US.variety.value_counts()

In [ ]:
US = US.groupby('variety').filter(lambda x: len(x) >500)

In [ ]:
wine_us =US.variety.unique().tolist()
wine_us.sort()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = US.drop(['Unnamed: 0','country','designation','points','province','taster_name',
       'taster_twitter_handle', 'title','region_1','region_2','variety','winery'], axis = 1)
y = US.variety
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
output = set()
for x in US.variety:
    x = x.lower()
    x = x.split()
    for y in x:
        output.add(y)
variety_list =sorted(output)

In [ ]:
extras = ['.', ',', '"', "'", '?', '!', ':', ';','-' ,'(', ')', '[', ']', '{', '}', 'cab',"%"]
from nltk.corpus import stopwords
stop = set(stopwords.words('english'))
stop.update(variety_list)
stop.update(extras)

In [ ]:
US.variety.value_counts()

In [ ]:
US.head()

In [ ]:
sum(US['year']>2002)

In [ ]:
datos_=US[US['year']>2002] 

In [ ]:
plt.figure(figsize=(10,4))
datos_.variety.value_counts().plot.bar()

In [ ]:
rep_v = datos_.groupby('variety')['year'].agg('median')

In [ ]:
for i in rep_v.index :
    datos_.loc[(datos_['year'].isna()) & (datos_['variety'] == i),'year']=rep_v[i]

In [ ]:
datos_.variety.describe()

In [ ]:
#data necesaria
#Tomando las provincias como dummy
X_ = datos_.drop(['Unnamed: 0','designation','country','taster_name',
       'taster_twitter_handle', 'title','region_1','region_2','variety','winery'], axis = 1)
X = pd.get_dummies(X_,columns=["province"])
y = datos_.variety

In [ ]:
X.info()

In [ ]:
X.head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 

In [ ]:
#generando variable de textmining
from scipy.sparse import hstack
vect = CountVectorizer(stop_words = stop)
X_dtm = vect.fit_transform(X.description)
Xs = X.drop(['description'],axis=1).as_matrix()
X_dtm = hstack((Xs,X_dtm))
X_dtm

In [ ]:
X_dtm.shape,X.shape

In [ ]:
len(y)

In [ ]:
Xtrain,Xtest,ytrain,ytest = train_test_split(X_dtm,y,random_state=1)

In [ ]:
wine=y.unique()

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
models = {}
for z in wine:
    model = LogisticRegression()
    y = ytrain == z
    model.fit(Xtrain, y)
    models[z] = model
testing_probs = pd.DataFrame(columns = wine)

In [ ]:
len(wine)

In [ ]:
probs = pd.DataFrame(columns = wine)

In [ ]:
for z in wine: 
    probs[z] = models[z].predict_proba(Xtest)[:,1]

In [ ]:
probs.head()

In [ ]:
probs_=probs.fillna(0)

In [ ]:
pred = probs.idxmax(axis=1)

In [ ]:
comparison = pd.DataFrame({'actual':ytest.values, 'predicted':pred.values})   

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score

In [ ]:
print('Accuracy Score:',accuracy_score(comparison.actual, comparison.predicted)*100,"%")

In [ ]:
comparison.head(50)

In [ ]:
from  sklearn.metrics import confusion_matrix

conf_mat = confusion_matrix(comparison.actual, comparison.predicted)
fig, ax = plt.subplots(figsize=(15,13))
sns.heatmap(conf_mat, annot=True, fmt='d')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
print (classification_report(comparison.actual, comparison.predicted))

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
models = {}
for z in wine:
    model = MultinomialNB()
    y = ytrain == z
    model.fit(Xtrain, y)
    models[z] = model
testing_probs = pd.DataFrame(columns = wine)

In [ ]:
probs = pd.DataFrame(columns = wine)

In [ ]:
for z in wine: 
    probs[z] = models[z].predict_proba(Xtest)[:,1]

In [ ]:
probs.head()


In [ ]:
probs_=probs.fillna(0)

In [ ]:
pred = probs.idxmax(axis=1)

In [ ]:
comparison = pd.DataFrame({'actual':ytest.values, 'predicted':pred.values})   

In [ ]:
print('Accuracy Score:',accuracy_score(comparison.actual, comparison.predicted)*100,"%")

In [ ]:
from sklearn import preprocessing 

In [ ]:
le  =  preprocessing.LabelEncoder() 

In [ ]:
le.fit(datos_['variety'])  

In [ ]:
le.transform(datos_['variety'])  

In [ ]:
datos_['var_codes'] = le.fit_transform(datos_['variety'])

In [ ]:
from sklearn import metrics

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(datos_['description'],datos_['var_codes'], test_size=0.33, random_state=42)

In [ ]:
modelo_cvec = CountVectorizer(stop_words=stop)

In [ ]:
X_train = modelo_cvec.fit_transform(X_train)

In [ ]:
modelo_NB = MultinomialNB()
modelo_NB.fit(X_train, y_train)

In [ ]:
# Predicción
X_test = modelo_cvec.transform(X_test)
labels_predichas = modelo_NB.predict(X_test)

In [ ]:
accuracy_score(y_test, labels_predichas)

In [ ]:
print (classification_report(y_test, labels_predichas))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model = RandomForestClassifier(n_estimators=1000)
model.fit(X_train, y_train)
ypred = model.predict(X_test)

In [ ]:
print(metrics.classification_report(ypred, y_test))

In [ ]:
#data necesaria
#sin dummies
X_ = datos_.drop(['Unnamed: 0','designation','country','taster_name','province',
       'taster_twitter_handle', 'title','region_1','region_2','variety','winery','var_codes'], axis = 1)

y = datos_.variety

In [ ]:
X_.info()

In [ ]:
#generando variable de textmining
from scipy.sparse import hstack
vect = CountVectorizer(stop_words = stop)
X_dtm = vect.fit_transform(X_.description)
Xs = X_.drop(['description'],axis=1).as_matrix()
X_dtm = hstack((Xs,X_dtm))
X_dtm

In [ ]:
Xtrain,Xtest,ytrain,ytest = train_test_split(X_dtm,y,random_state=1)

In [ ]:
wine=y.unique()

In [ ]:
models = {}
for z in wine:
    model = LogisticRegression()
    y = ytrain == z
    model.fit(Xtrain, y)
    models[z] = model
testing_probs = pd.DataFrame(columns = wine)

In [ ]:
probs = pd.DataFrame(columns = wine)

In [ ]:
for z in wine: 
    probs[z] = models[z].predict_proba(Xtest)[:,1]

In [ ]:
probs.head()

In [ ]:
probs_=probs.fillna(0)

In [ ]:
pred = probs.idxmax(axis=1)

In [ ]:
comparison = pd.DataFrame({'actual':ytest.values, 'predicted':pred.values})   

In [ ]:
print('Accuracy Score:',accuracy_score(comparison.actual, comparison.predicted)*100,"%")

In [ ]:
conf_mat = confusion_matrix(comparison.actual, comparison.predicted)
fig, ax = plt.subplots(figsize=(15,13))
sns.heatmap(conf_mat, annot=True, fmt='d')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

In [ ]:
import re
import nltk

In [ ]:
import string
# Importando todo NLTK
import nltk

# Tambien se puede importar modulos
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer

#Downloading NLP corpus from NLTK 
nltk.download('stopwords')
nltk.download('wordnet')

In [ ]:
# Stop words = Creacion de un conjunto unico desde el listado de stopwords en ingles que viene con el paquete NLTK. 

#stop = set(stopwords.words('english'))

# Idem punto anterior, se excluyen signos de puntuacion 

exclude = set(string.punctuation) 

# Lemmatizacion de las palabras 
lemma = WordNetLemmatizer()

def clean(doc):
    #pasar a minusculas separando por espacios
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

In [ ]:
datos_.variety.value_counts()

In [ ]:
wine_data=datos_.groupby('variety')

In [ ]:
wine_data.get_group('Red Blend')

In [ ]:
#bORRAMOS ALGUNAS PALABRAS MAS COMUNES EN LA DESCRIPTION
#borramos los nombres de las variety en la description
def clean_text(text):
    text = re.sub('rose', ' ', text)
    text = re.sub('cabernet sauvignon', ' ', text)
    text = re.sub('syrah', ' ', text)
    text = re.sub('red blend', ' ', text)
    text = re.sub('zinfandel', ' ', text)
    text = re.sub('sauvignon blanc', ' ', text)
    text = re.sub('bordeaux-style Red Blend', ' ', text)
    text = re.sub('zinfandel', ' ', text)
    text = re.sub('riesling', ' ', text)
    text = re.sub('cabernet franc', ' ', text)
    text = re.sub('merlot', ' ', text)
    text = re.sub('the', ' ', text)
    text = re.sub('wine', ' ', text)
    text = re.sub('drink', ' ', text)
    text = re.sub('acidity', ' ', text)
    text = re.sub('aroma', ' ', text)
    text = re.sub('aromas', ' ', text)
    text = re.sub('finish', ' ', text)
    text = re.sub('fruit', ' ', text)
    text = re.sub('palate', ' ', text)
    text = re.sub('sparkling blend', ' ', text)
    text = re.sub('pinot gris', ' ', text)
    text = re.sub('riesling', ' ', text)
    text = re.sub('petite sirah', ' ', text)

    return text

In [ ]:
modelo_cvec = CountVectorizer(stop_words=stop)

In [ ]:
datos_['description'] = datos_['description'].map(lambda com : clean(com))

In [ ]:
datos_['description'] = datos_['description'].map(lambda com : clean_text(com))

In [ ]:
datos_['description']= datos_['description'].apply(lambda x: re.sub(r'([^\s\w]|_)+', '', x))

In [ ]:
datos_['variety'].value_counts(normalize=True)

In [ ]:
datos_.shape

In [ ]:
Cons = datos_['description']

In [ ]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer(min_df=0.05, max_df=0.9,stop_words=stop)
data_vectorized = vectorizer.fit_transform(Cons)

In [ ]:
tf_feature_names = vectorizer.get_feature_names()

In [ ]:
lda_model = LatentDirichletAllocation(n_topics=10, max_iter=30, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)

In [ ]:
import pyLDAvis.sklearn
 
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel

In [ ]:
lda_fit = lda_model.fit(data_vectorized)

In [ ]:
# Ejecutamos un método de visualización de los tópicos

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from __future__ import print_function

In [ ]:
pyLDAvis.save_html(panel, 'lda_topics_US10.html')

In [ ]:
Cons = datos_['description']

In [ ]:
# Guardamos en una nueva serie

Cons_clean = [clean(comment).split() for comment in Cons] 


In [ ]:
from gensim import corpora, models 
import gensim

In [ ]:
# converting the corpus into a document-term matrix. every unique term is assigned an index.

dictionary = corpora.Dictionary(Cons_clean)

In [ ]:
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above. Usamos doc2bow de gensim

doc_term_matrix = [dictionary.doc2bow(comment) for comment in Cons_clean]

In [ ]:
X_lda = pd.DataFrame(lda_Z)

In [ ]:
df_lda = X_lda.copy()

In [ ]:
df_lda['Topico'] = df_lda.idxmax(axis=1)

In [ ]:
df_lda.head()

In [ ]:
lda_model = LatentDirichletAllocation(n_topics=20, max_iter=30, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)

In [ ]:
pyLDAvis.enable_notebook()
panel20 = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, mds='tsne')
panel20

In [ ]:
lda_fit = lda_model.fit(data_vectorized)